In [1]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import json
import os

pd.set_option('future.no_silent_downcasting', True)

In [2]:
airports = pd.read_csv('C:/Users/Николай/PycharmProjects/FlightRank_2025/data/airports.csv')

In [3]:
airports1 = pd.read_csv('C:/Users/Николай/PycharmProjects/FlightRank_2025/data/airports (1).csv')

In [4]:
train = pd.read_parquet('C:/Users/Николай/PycharmProjects/FlightRank_2025/data/train.parquet', engine='pyarrow',
                        columns=['legs0_segments0_departureFrom_airport_iata',
                                 'legs0_segments1_departureFrom_airport_iata',
                                 'legs0_segments2_departureFrom_airport_iata',
                                 'legs0_segments3_departureFrom_airport_iata',
                                 'legs1_segments0_departureFrom_airport_iata',
                                 'legs1_segments1_departureFrom_airport_iata',
                                 'legs1_segments2_departureFrom_airport_iata',
                                 'legs1_segments3_departureFrom_airport_iata',
                                 
                                 'legs0_segments0_arrivalTo_airport_iata',
                                 'legs0_segments1_arrivalTo_airport_iata',
                                 'legs0_segments2_arrivalTo_airport_iata',
                                 'legs0_segments3_arrivalTo_airport_iata',
                                 'legs1_segments0_arrivalTo_airport_iata',
                                 'legs1_segments1_arrivalTo_airport_iata',
                                 'legs1_segments2_arrivalTo_airport_iata',
                                 'legs1_segments3_arrivalTo_airport_iata',
                                 ])

test = pd.read_parquet('C:/Users/Николай/PycharmProjects/FlightRank_2025/data/test.parquet', engine='pyarrow',
                        columns=['legs0_segments0_departureFrom_airport_iata',
                                 'legs0_segments1_departureFrom_airport_iata',
                                 'legs0_segments2_departureFrom_airport_iata',
                                 'legs0_segments3_departureFrom_airport_iata',
                                 'legs1_segments0_departureFrom_airport_iata',
                                 'legs1_segments1_departureFrom_airport_iata',
                                 'legs1_segments2_departureFrom_airport_iata',
                                 'legs1_segments3_departureFrom_airport_iata',
                                 
                                 'legs0_segments0_arrivalTo_airport_iata',
                                 'legs0_segments1_arrivalTo_airport_iata',
                                 'legs0_segments2_arrivalTo_airport_iata',
                                 'legs0_segments3_arrivalTo_airport_iata',
                                 'legs1_segments0_arrivalTo_airport_iata',
                                 'legs1_segments1_arrivalTo_airport_iata',
                                 'legs1_segments2_arrivalTo_airport_iata',
                                 'legs1_segments3_arrivalTo_airport_iata',
                                 ])

In [5]:
del airports['AirportName']
del airports['ICAO']
del airports['TimeZone']
del airports['City_IATA']
del airports['UTC_Offset_Seconds']
del airports['Country_CodeA2']
del airports['Country_CodeA3']
del airports['GeoPointLat']
del airports['GeoPointLong']
del airports['UTC_Offset_Hours']
airports = airports.dropna(subset=['IATA'])

del airports1['id']
del airports1['ident']
del airports1['name']
del airports1['iso_country']
del airports1['scheduled_service']
del airports1['icao_code']
del airports1['gps_code']
del airports1['local_code']
del airports1['home_link']
del airports1['wikipedia_link']
del airports1['keywords']
del airports1['elevation_ft']
del airports1['iso_region']
del airports1['municipality']

In [6]:
iata_unique = set()
for col in train.columns:
    iata_unique.update(train[col].dropna().unique())
for col in test.columns:
    iata_unique.update(test[col].dropna().unique())

airports = airports[airports['IATA'].isin(iata_unique)]

In [10]:
airports = airports.merge(
    airports1,
    left_on='IATA',
    right_on='iata_code',
    how='left'
)
del airports['iata_code']

In [19]:
fill_values = {
    'DFW': {'continent': 'NA'},
    'XIK': {'latitude_deg': 45.29, 'longitude_deg': 9.12, 'continent': 'EU'},
    'IMR': {'latitude_deg': 45.43, 'longitude_deg': 9.23, 'continent': 'EU'},
    'QKL': {'latitude_deg': 50.94, 'longitude_deg': 6.95, 'continent': 'EU'},
    'QKU': {'latitude_deg': 50.94, 'longitude_deg': 6.97, 'continent': 'EU'},
    'ZYR': {'latitude_deg': 50.5, 'longitude_deg': 4.20, 'continent': 'EU'},
    'DMS': {'latitude_deg': 26.47, 'longitude_deg': 49.79, 'continent': 'AS'},
    'ZLP': {'latitude_deg': 47.27, 'longitude_deg': 8.32, 'continent': 'EU'},
    'XVQ': {'latitude_deg': 45.26, 'longitude_deg': 12.19, 'continent': 'EU'},
    'XVY': {'latitude_deg': 45.30, 'longitude_deg': 12.21, 'continent': 'EU'},
    'PTY': {'continent': 'NA'},
    'YJB': {'latitude_deg': 41.38, 'longitude_deg': 2.14, 'continent': 'EU'},
    'ZYA': {'latitude_deg': 52.37, 'longitude_deg': 4.9, 'continent': 'EU'},
    'CUN': {'continent': 'NA'},
    'YUL': {'continent': 'NA'},
    'SAN': {'continent': 'NA'},
    'NHR': {'latitude_deg': 40.53, 'longitude_deg': 14.17, 'continent': 'EU'},
    'INP': {'latitude_deg': 40.85, 'longitude_deg': 14.27, 'continent': 'EU'},
    'ZMU': {'latitude_deg': 48.35, 'longitude_deg': 11.79, 'continent': 'EU'},
    'QQM': {'latitude_deg': 53.5, 'longitude_deg': -2.25, 'continent': 'EU'},
    'XWC': {'latitude_deg': 48.15, 'longitude_deg': 16.25, 'continent': 'EU'},
    'BOS': {'continent': 'NA'},
    'IBT': {'latitude_deg': 44.32, 'longitude_deg': 44.53, 'continent': 'EU'},
    'MTY': {'continent': 'NA'},
    'ZBA': {'latitude_deg': 47.35, 'longitude_deg': 7.31, 'continent': 'EU'},
    'SJO': {'continent': 'NA'},
    'IAD': {'continent': 'NA'},
    'YJV': {'latitude_deg': 39.48, 'longitude_deg': -0.48, 'continent': 'EU'},
    'HAV': {'continent': 'NA'},
    'LGA': {'continent': 'NA'},
    'JFK': {'continent': 'NA'},
    'EWR': {'continent': 'NA'},
    'ZVR': {'latitude_deg': 52.37, 'longitude_deg': 9.73, 'continent': 'EU'},
    'YYZ': {'continent': 'NA'},
    'MIA': {'continent': 'NA'},
    'OAX': {'continent': 'NA'},
    'IAH': {'continent': 'NA'},
    'SJU': {'continent': 'NA'},
    'XTI': {'latitude_deg': 40.47, 'longitude_deg': -3.68, 'continent': 'EU'},
    'XOC': {'latitude_deg': 40.41, 'longitude_deg': -3.7, 'continent': 'EU'},
    'QDU': {'latitude_deg': 51.22, 'longitude_deg': 6.79, 'continent': 'EU'},
    'VRA': {'continent': 'NA'},
    'ZDN': {'latitude_deg': 49.09, 'longitude_deg': 16.41, 'continent': 'EU'},
    'ZAQ': {'latitude_deg': 49.29, 'longitude_deg': 11.04, 'continent': 'EU'},
    'ZWS': {'latitude_deg': 48.41, 'longitude_deg': 09.13, 'continent': 'EU'},
    'IRT': {'latitude_deg': 41.48, 'longitude_deg': 12.14, 'continent': 'EU'},
    'XRJ': {'latitude_deg': 41.47, 'longitude_deg': 12.35, 'continent': 'EU'},
    'SAL': {'continent': 'NA'},
    'MCO': {'continent': 'NA'},
    'ATL': {'continent': 'NA'},
    'DEN': {'continent': 'NA'},
    'PHX': {'continent': 'NA'},
    'ORD': {'continent': 'NA'},
    'ZVJ': {'latitude_deg': 24.25, 'longitude_deg': 54.39, 'continent': 'AS'},
    'BUR': {'continent': 'NA'},
    'HHL': {'latitude_deg': 40.28, 'longitude_deg': 111.34, 'continent': 'AS'},
    'HZD': {'latitude_deg': 35.03, 'longitude_deg': 115.23, 'continent': 'AS'},
    'DTW': {'continent': 'NA'},
    'ZSB': {'latitude_deg': 47.47, 'longitude_deg': 13.0, 'continent': 'EU'},
    'XXQ': {'latitude_deg': 47.3, 'longitude_deg': 19.05, 'continent': 'EU'},
    'SEA': {'continent': 'NA'},
    'XYD': {'latitude_deg': 45.76, 'longitude_deg': 4.83, 'continent': 'EU'},
    'SFO': {'continent': 'NA'},
    'XYG': {'latitude_deg': 50.4, 'longitude_deg': 14.26, 'continent': 'EU'},
    'LAS': {'continent': 'NA'},
    'CLT': {'continent': 'NA'},
    'YVR': {'continent': 'NA'},
    'ONT': {'continent': 'NA'},
    'ZMB': {'latitude_deg': 53.37, 'longitude_deg': 09.59, 'continent': 'EU'},
    'XNB': {'latitude_deg': 25.25, 'longitude_deg': 55.33, 'continent': 'AS'},
    'ZDH': {'latitude_deg': 47.35, 'longitude_deg': 07.31, 'continent': 'EU'},
    'MLH': {'latitude_deg': 47.35, 'longitude_deg': 07.31, 'continent': 'EU'},
    'MEX': {'continent': 'NA'},
    'NLU': {'continent': 'NA'},
    'QPP': {'latitude_deg': 52.52, 'longitude_deg': 13.37, 'continent': 'EU'},
    'PHL': {'continent': 'NA'},
    'LAX': {'continent': 'NA'},
    'AGY': {'latitude_deg': 48.25, 'longitude_deg': 48.42, 'continent': 'EU'},
    'HOU': {'continent': 'NA'},
    'ZFQ': {'latitude_deg': 44.83, 'longitude_deg': -0.56, 'continent': 'EU'},
    'SJC': {'continent': 'NA'},
    'SNA': {'continent': 'NA'},
    'QLS': {'latitude_deg': 46.54, 'longitude_deg': 6.61, 'continent': 'EU'},
    'MSP': {'continent': 'NA'},
    'DHC': {'latitude_deg': 53.04, 'longitude_deg': 53.02, 'continent': 'EU'},
    'WUB': {'latitude_deg': 31.29, 'longitude_deg': 31.49, 'continent': 'AS'},
    'XIX': {'latitude_deg': 45.23, 'longitude_deg': 10.53, 'continent': 'EU'},
    'YJM': {'latitude_deg': 36.675, 'longitude_deg': -4.499, 'continent': 'EU'},
    'QRO': {'continent': 'NA'},
}

for iata, values in fill_values.items():
    idx = airports[airports['IATA'] == iata].index
    if not idx.empty:
        for col, val in values.items():
            airports.at[idx[0], col] = val

In [21]:
airports['type'].fillna('RAILWAY', inplace=True)

C:\Users\Николай\AppData\Local\Temp\ipykernel_5276\125646077.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  airports['type'].fillna('RAILWAY', inplace=True)


In [23]:
airports['IATA'] = airports['IATA'].astype('category')
airports['City_Name'] = airports['City_Name'].astype('category')
airports['Country_Name'] = airports['Country_Name'].astype('category')
airports['type'] = airports['type'].astype('category')
airports['latitude_deg'] = airports['latitude_deg'].round()
airports['longitude_deg'] = airports['longitude_deg'].round()
airports['latitude_deg'] = airports['latitude_deg'].astype('int16')
airports['longitude_deg'] = airports['longitude_deg'].astype('int16')
airports['continent'] = airports['continent'].astype('category')

In [25]:
legs0_segments0_arrivalTo_airport_iata_map = {old_id: new_id for new_id, old_id in enumerate(train['legs0_segments0_arrivalTo_airport_iata'].unique())}
legs0_segments1_arrivalTo_airport_iata_map = {old_id: new_id for new_id, old_id in enumerate(train['legs0_segments1_arrivalTo_airport_iata'].unique())}
legs0_segments2_arrivalTo_airport_iata_map = {old_id: new_id for new_id, old_id in enumerate(train['legs0_segments2_arrivalTo_airport_iata'].unique())}
legs0_segments3_arrivalTo_airport_iata_map = {old_id: new_id for new_id, old_id in enumerate(train['legs0_segments3_arrivalTo_airport_iata'].unique())}

legs1_segments0_arrivalTo_airport_iata_map = {old_id: new_id for new_id, old_id in enumerate(train['legs1_segments0_arrivalTo_airport_iata'].unique())}
legs1_segments1_arrivalTo_airport_iata_map = {old_id: new_id for new_id, old_id in enumerate(train['legs1_segments1_arrivalTo_airport_iata'].unique())}
legs1_segments2_arrivalTo_airport_iata_map = {old_id: new_id for new_id, old_id in enumerate(train['legs1_segments2_arrivalTo_airport_iata'].unique())}
legs1_segments3_arrivalTo_airport_iata_map = {old_id: new_id for new_id, old_id in enumerate(train['legs1_segments3_arrivalTo_airport_iata'].unique())}

legs0_segments0_departureFrom_airport_iata_map = {old_id: new_id for new_id, old_id in enumerate(train['legs0_segments0_departureFrom_airport_iata'].unique())}
legs0_segments1_departureFrom_airport_iata_map = {old_id: new_id for new_id, old_id in enumerate(train['legs0_segments1_departureFrom_airport_iata'].unique())}
legs0_segments2_departureFrom_airport_iata_map = {old_id: new_id for new_id, old_id in enumerate(train['legs0_segments2_departureFrom_airport_iata'].unique())}
legs0_segments3_departureFrom_airport_iata_map = {old_id: new_id for new_id, old_id in enumerate(train['legs0_segments3_departureFrom_airport_iata'].unique())}

legs1_segments0_departureFrom_airport_iata_map = {old_id: new_id for new_id, old_id in enumerate(train['legs1_segments0_departureFrom_airport_iata'].unique())}
legs1_segments1_departureFrom_airport_iata_map = {old_id: new_id for new_id, old_id in enumerate(train['legs1_segments1_departureFrom_airport_iata'].unique())}
legs1_segments2_departureFrom_airport_iata_map = {old_id: new_id for new_id, old_id in enumerate(train['legs1_segments2_departureFrom_airport_iata'].unique())}
legs1_segments3_departureFrom_airport_iata_map = {old_id: new_id for new_id, old_id in enumerate(train['legs1_segments3_departureFrom_airport_iata'].unique())}

In [26]:
maps = {
    "legs0_segments0_arrivalTo_airport_iata_map": legs0_segments0_arrivalTo_airport_iata_map,
    "legs0_segments1_arrivalTo_airport_iata_map": legs0_segments1_arrivalTo_airport_iata_map,
    "legs0_segments2_arrivalTo_airport_iata_map": legs0_segments2_arrivalTo_airport_iata_map,
    "legs0_segments3_arrivalTo_airport_iata_map": legs0_segments3_arrivalTo_airport_iata_map,
    
    "legs1_segments0_arrivalTo_airport_iata_map": legs1_segments0_arrivalTo_airport_iata_map,
    "legs1_segments1_arrivalTo_airport_iata_map": legs1_segments1_arrivalTo_airport_iata_map,
    "legs1_segments2_arrivalTo_airport_iata_map": legs1_segments2_arrivalTo_airport_iata_map,
    "legs1_segments3_arrivalTo_airport_iata_map": legs1_segments3_arrivalTo_airport_iata_map,
    
    "legs0_segments0_departureFrom_airport_iata_map": legs0_segments0_departureFrom_airport_iata_map,
    "legs0_segments1_departureFrom_airport_iata_map": legs0_segments1_departureFrom_airport_iata_map,
    "legs0_segments2_departureFrom_airport_iata_map": legs0_segments2_departureFrom_airport_iata_map,
    "legs0_segments3_departureFrom_airport_iata_map": legs0_segments3_departureFrom_airport_iata_map,
    
    "legs1_segments0_departureFrom_airport_iata_map": legs1_segments0_departureFrom_airport_iata_map,
    "legs1_segments1_departureFrom_airport_iata_map": legs1_segments1_departureFrom_airport_iata_map,
    "legs1_segments2_departureFrom_airport_iata_map": legs1_segments2_departureFrom_airport_iata_map,
    "legs1_segments3_departureFrom_airport_iata_map": legs1_segments3_departureFrom_airport_iata_map
}

In [28]:
current_dir = os.getcwd()
folder_name = os.path.basename(current_dir)

airports_path = f"{folder_name}_airports.csv"
airports.to_csv(airports_path, index=False)

with open(f"{folder_name}_airport_iata_maps.json", "w", encoding="utf-8") as f:
    json.dump(maps, f, ensure_ascii=False, indent=2)

In [29]:
airports

,IATA,City_Name,Country_Name,type,latitude_deg,longitude_deg,continent
0,NRT,Tokyo,Japan,large_airport,36,140,AS
1,HND,Tokyo,Japan,large_airport,36,140,AS
2,VNS,Varanasi,India,medium_airport,25,83,AS
3,BAV,Baotou,China,medium_airport,41,110,AS
4,BMV,Banmethuot,Viet Nam,medium_airport,13,108,AS
...,...,...,...,...,...,...,...
686,WDS,Shiyan,China,medium_airport,33,111,AS
687,BZX,BAZHONG,China,small_airport,32,107,AS
688,NBS,Baishan,China,medium_airport,42,128,AS
689,EYK,Beloyarsky (HMAO),Russian Federation,medium_airport,64,67,EU
